INSTALL DEPENDENCIES

In [1]:
'''
pip install ydata-profiling
pip install ipywidgets
'''

'\npip install ydata-profiling\npip install ipywidgets\n'

IMPORT LIBRARIES

In [20]:
import pandas as pd
import configparser
import snowflake.connector
from ydata_profiling import ProfileReport
from pandas_profiling import ProfileReport

C:\Users\a.canod\AppData\Local\Temp\ipykernel_21028\2424073424.py:5: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


CONFIG CONNECTION TO SNOWFLAKE

In [2]:
config = configparser.ConfigParser()
config.read(r"settings_95592.ini")

['settings_95592.ini']

CONNECT TO DB IN SNOWFLAKE

In [3]:
try:
    conn = snowflake.connector.connect(
        account=config["snowflake"]["account"],
        user=config["snowflake"]["user"],
        password=config["snowflake"]["password"],
        database=config["snowflake"]["database"],
        warehouse=config["snowflake"]["warehouse"],
        schema=config["snowflake"]["schema"]
    )
    print('Connected to SF')
except Exception as e:
        print(f"Error connecting to Snowflake: {str(e)}")


Connected to SF


OPEN CONNECTION

In [4]:
cursor = conn.cursor()

QUERY THE DB

In [17]:
query = '''

with demographics as(
select distinct
// Demographics
    p.patientid
    ,c.claimid
    ,de.departmentname
    ,de.facilityid    
    ,p.firstname
    ,p.lastname
    ,p.namesuffix
    ,p.patientssn
    ,cast(p.dob as Date) as dob
    ,p.address
    ,p.address2
    ,p.city
    ,p.state
    ,p.zip
    ,p.patienthomephone
    ,p.mobilephone
    ,p.workphone
    ,e.name as "Patient employer"
    ,p.genderidentity
    ,p.deceaseddate
    ,cast(vi.checkindatedatetime as Date) as "Admit date"
    ,cast(vi.dischargedatedatetime as Date) as "Discharge date"
    ,pay.insurancepackagename
    ,pay.ircgroup
    ,pp.vendorcode
    ,concat(concat(pv.providerfirstname,' '),pv.providerlastname) as "Provider full name"
    
  
from athenahealth.athenaone.claim as c
left join athenahealth.athenaone.patient as p on p.patientid = c.patientid
left join athenahealth.athenaone.department as de on c.servicedepartmentid = de.departmentid
left join athenahealth.athenaone.employer as e on p.patientemployerid = e.employerid
left join athenahealth.athenaone.patientroundinglist as pr on pr.patientid = p.patientid
left join athenahealth.athenaone.patientinsurance as pa on pa.patientinsuranceid = c.claimprimarypatientinsid
left join athenahealth.athenaone.payer as pay on pay.insurancepackageid = pa.insurancepackageid
left join athenahealth.athenaone.patientpayment as pp on pp.claimid = c.claimid
left join athenahealth.athenaone.provider as pv on pv.providerid = c.primarybillingproviderid
left join athenahealth.athenaone.visitbillvisit as v on c.visitbillid = v.visitbillid
left join athenahealth.athenaone.visit as vi on v.visitid = vi.visitid

where
    c.claimservicedate >='2019-1-1'

    
order by
    p.firstname
    ,p.lastname
    ,c.claimid
),
financials as(
select 
    af.claimid
    ,sum(case when transfertype in ('1', '2') and type = 'PAYMENT' then amount end) as "Insurance payment"
    ,sum(case when transfertype = 'p' and type = 'PAYMENT' then amount end) as "Patient payment"     
    ,sum(case when transfertype in ('1', '2') and type ='ADJUSTMENT' then amount end) as "Insurance adjustment"
    ,sum(case when transfertype = 'p' and type ='ADJUSTMENT' then amount end) as "Patient adjustment"
    ,sum(case when type ='CHARGE' then amount end) as "Charges"
    ,sum(case when type in ('PAYMENT', 'ADJUSTMENT','CHARGE') then amount end) as "Total amount remaining"
    
from athenahealth.financials.activityfact as af
left join athenahealth.athenaone.claim as c on c.claimid = af.claimid

group by 
    af.claimid
)
select
    d.patientid
    ,d.claimid
    ,d.departmentname
    ,d.facilityid    
    ,d.firstname
    ,d.lastname
    ,d.namesuffix
    ,d.patientssn
    ,d.dob
    ,d.address
    ,d.address2
    ,d.city
    ,d.state
    ,d.zip
    ,d.patienthomephone
    ,d.mobilephone
    ,d.workphone
    ,d."Patient employer"
    ,d.genderidentity
    ,d.deceaseddate
    ,d."Admit date"
    ,d."Discharge date"
    ,d.insurancepackagename
    ,d.ircgroup
    ,fi."Charges"
    ,fi."Insurance payment"
    ,fi."Patient payment" 
    ,fi."Insurance adjustment"
    ,fi."Patient adjustment"
    ,fi."Total amount remaining"
    ,d.vendorcode
    ,d."Provider full name"
    
from demographics as d
left join financials as fi on fi.claimid = d.claimid


'''

cursor.execute(query)

try:
# Get data
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description] 

    # Create df
    df = pd.DataFrame(data, columns=column_names)

    print('df extracted from SF')

except Exception as e:
        print(f"Error: {str(e)}")

df extracted from SF


EDA FOR LOP

In [30]:
df['DOB'] = pd.to_datetime(df['DOB']).dt.to_pydatetime()
df['Admit date'] = pd.to_datetime(df['Admit date']).dt.to_pydatetime()
df['Discharge date'] = pd.to_datetime(df['Discharge date']).dt.to_pydatetime()

profile = ProfileReport(df, title="Profiling Report for Previous month total charges Analysis", minimal=True)
profile.to_notebook_iframe()

C:\Users\a.canod\AppData\Local\Temp\ipykernel_21028\4115386207.py:1: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  df['DOB'] = pd.to_datetime(df['DOB']).dt.to_pydatetime()
C:\Users\a.canod\AppData\Local\Temp\ipykernel_21028\4115386207.py:2: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  df['Admit date'] = pd.to_datetime(df['Admit date']).dt.to_pydatetime()
C:\Users\a.canod\AppData\Local\Temp\ipykernel_21028\4115386207.py:3: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarr

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

AttributeError: 'datetime.date' object has no attribute 'hour'

CLOSE CONNECTION

In [8]:
conn.close()